In [ ]:
from mycode.common import *

构造数据

In [ ]:
metadata, tables = fetch_data_from_sqlite(path='./mycode/data_sqlite.db')
metadata = sdvMetadata(metadata)
metadata.get_tables()
result_table = tables["BookLoan"]
x_table = ["Book", "Library", "Student", "Enrollment", "Submission"]
x_key = ['book_id', "library_id", "student_id", "student_id","student_id"]
x_how = ['inner' for i in range(5)]
print(f"{len(result_table)=}")
for i, x_table_name in enumerate(x_table):
    result_table = pd.merge(result_table, tables[x_table_name], on=x_key[i], how=x_how[i])
    print(f"{len(result_table)}", end=" ")
print(len(result_table.columns))
#result_table

In [ ]:
# result_table = result_table[:100]
result_table

检查metadata

In [ ]:
result_table.to_csv("test_10k_label.csv", index=False)
dataset_csv = "test_10k_label.csv"
data_connector = CsvConnector(path=dataset_csv)
data_loader = DataLoader(data_connector)
loan_metadata = Metadata.from_dataloader(data_loader)

loan_metadata.discrete_columns

In [ ]:
loan_metadata.datetime_columns

In [ ]:
loan_metadata.column_list.__len__()

In [ ]:
loan_metadata.datetime_format = {
    key: "%Y-%m-%d" for key in loan_metadata.datetime_columns if key != "submission_date" 
}
loan_metadata.datetime_format["submission_date"] = "%Y-%m-%d %H:%M:%S"
loan_metadata.datetime_format

In [ ]:
loan_metadata.discrete_columns = set([
    key for key in loan_metadata.discrete_columns if key not in loan_metadata.datetime_columns
])
# loan_metadata.label_columns = loan_metadata.discrete_columns
#loan_metadata.label_columns.add("grade_x")
#loan_metadata.label_columns.add("grade_y")
print(f"{loan_metadata.int_columns=}\n{loan_metadata.float_columns=}\n{loan_metadata.const_columns=}\n{loan_metadata.bool_columns=}\n{loan_metadata.discrete_columns=}")

In [ ]:
loan_metadata.column_encoder = {
    key: "label" for key in loan_metadata.discrete_columns
}

训练

In [ ]:


ctgan = CTGANSynthesizerModel(
        epochs=1,
        batch_size=500,
        # device="cpu"
    )
# Initialize synthesizer, use CTGAN model
synthesizer = Synthesizer(
    metadata=loan_metadata,
    model=ctgan,  # For quick demo
    data_connector=data_connector,
)


writetime()
synthesizer.fit()
writetime()
#synthesizer.save("test_model_10k_id10")

In [ ]:
data_res = synthesizer.sample(900)

In [ ]:
data_res